In [ ]:
import pandas as pd
import json
import os
import ast
import re

In [ ]:
LOCATIE_REQUESTS = os.path.join("../data/requests")
RUWE_DATA_CSV = os.path.join('../data/ruw/all_ruwe_data.csv')
GEKREGEN_EXCEL_FILE = os.path.join('../data/ModifiedQueryRows.xlsx')

In [ ]:
def parse_filename(file_name) -> tuple:
    try:
        # rsplit verwijdert alleen de laatste .json extensie (voor mocht het bestand meerdere .json bevatten alhoewel dit onwaarschijnlijk lijkt)
        parts = file_name.rsplit('.json', 1)[0].split('-')
        if len(parts) >= 5:
            route_id = parts[0]
            date = parts[1]
            time = parts[2]
            number_of_tasks = parts[3]
            number_of_tasks_in_input_plan = parts[4]
            return route_id, date, time, number_of_tasks, number_of_tasks_in_input_plan
        else:
            print(f"Skipping file {file_name}: incorrect format")
            return None
    except Exception as e:
        print(f"Error parsing file name {file_name}: {e}")
        return None

In [ ]:
# check if csv bestand bestaat
def check_csv_bestand(bestandspadennaam) -> bool:
    if os.path.exists(bestandspadennaam):
        return True
    return False

# lees csv bestand in als dataframe
def lees_dataframe_uit_csv(bestandspadennaam) -> pd.DataFrame:
    df = pd.read_csv(bestandspadennaam)  
    return df

# maak csv bestand aan vanuit dataframe
def schrijf_dataframe_naar_csv(df: pd.DataFrame, bestandspadennaam) -> None:
    # Maak de directory aan als deze niet bestaat
    os.makedirs(os.path.dirname(bestandspadennaam), exist_ok=True)
    # Schrijf het dataframe naar CSV
    df.to_csv(bestandspadennaam, index=False)
    return None

# de verschillende json-bestanden van de requests inladen en samenvoegen tot 1 grote dataframe 
# geeft een dataframe terug als resultaat
def lees_json_bestanden_en_maak_dataframe(locatie_requests) -> pd.DataFrame:
    df = pd.DataFrame()
    for folder_name in os.listdir(locatie_requests):
        folder_path = os.path.join(locatie_requests, folder_name)
        if os.path.isdir(folder_path):
            for file_name in os.listdir(folder_path):
                if file_name.endswith('.json'):
                    parsed_data = parse_filename(file_name)
                    if parsed_data == None:
                        break
                    else:
                        route_id, date, time, number_of_tasks, number_of_tasks_in_input_plan = parsed_data
                        # print(route_id, date, time, number_of_tasks, number_of_tasks_in_input_plan)
                        file_path = os.path.join(folder_path, file_name)
                        with open(file_path, 'r') as f:
                            data = json.load(f)
                            # Voeg route_id toe aan elke record in data
                            data['route_id'] = route_id
                            data['date'] = date
                            data['time'] = time
                            data['number_of_tasks'] = number_of_tasks
                            data['number_of_tasks_in_input_plan'] = number_of_tasks_in_input_plan
                            temp_df = pd.DataFrame([data])
                            # voeg tijdelijke dataframe toe aan de hoofddataframe
                            df = pd.concat([df, temp_df], ignore_index=True)
    
    return df


In [ ]:

if (check_csv_bestand(RUWE_DATA_CSV) == False):
    # maak het bestand een eerste keer

    ingelezen_dataframe = lees_json_bestanden_en_maak_dataframe(LOCATIE_REQUESTS)
    # dit is dan dezelfde dataframe als in de gegeven excel file maar met tasks en fixedTasks eraan toegevoegd en zonder TriggerType
    # date en time zijn nog steeds strings, kan later nog omgezet worden naar datetime indien nodig
    # timecalculation, 
    display(ingelezen_dataframe.head())

    schrijf_dataframe_naar_csv(ingelezen_dataframe, RUWE_DATA_CSV)

    display(ingelezen_dataframe.tail())
    print(f"Lengte van de dataframe: {len(ingelezen_dataframe)}")
    # excel inlezen om de lengte te checken
    df_excel = pd.read_excel(GEKREGEN_EXCEL_FILE)
    print(f"Aantal rijen in ingelezen excel dataframe: {len(df_excel)}")
    if (len(ingelezen_dataframe) != len(df_excel)):
        print("Waarschuwing: Aantal rijen in ingelezen dataframe komt niet overeen met aantal rijen in excel dataframe!")

else:
    # lees het bestand en voeg toe aan dataframe om verder mee te werken
    ingelezen_dataframe = lees_dataframe_uit_csv(RUWE_DATA_CSV)
    display(ingelezen_dataframe.tail())


In [ ]:
# dataframe met tasks maken
taken_df = ingelezen_dataframe[['id', 'tasks']].copy()
# type van tasks is een string dus moeten we dit eerst omzetten
def safe_eval(x):
    """Veilige evaluatie van lijststrings, werkt ook als pd.isna vastloopt."""
    # Eerst basischecks
    if x is None or x == '':
        return []

    # Dan pas pd.isna (werkt voor echte NaN waarden)
    try:
        if pd.isna(x):
            return []
    except Exception:
        pass  # Als pd.isna niet kan, gewoon doorgaan

    # Als het al een lijst is
    if isinstance(x, list):
        return x

    # Als het een string is, probeer te evalueren
    if isinstance(x, str):
        try:
            import ast
            return ast.literal_eval(x)
        except Exception:
            return []

    # Alles wat overblijft → lege lijst
    return []

# def safe_eval_no_ast(x):
#     """Veilige conversie van string naar lijst zonder ast.literal_eval."""
#     if not x or pd.isna(x):
#         return []

#     # Alleen lijsten herkennen: iets dat begint met [ en eindigt met ]
#     x = x.strip()
#     if x.startswith('[') and x.endswith(']'):
#         # Items splitsen op komma, strip aanhalingstekens en spaties
#         items = re.findall(r"(?:'([^']*)'|\"([^\"]*)\"|([^,\[\]]+))", x)
#         result = []
#         for t in items:
#             # t is een tuple van drie, één element is gevuld
#             value = next(filter(None, t))
#             value = value.strip()
#             # Probeer int of float te maken, anders als string
#             if value.isdigit():
#                 result.append(int(value))
#             else:
#                 try:
#                     result.append(float(value))
#                 except ValueError:
#                     result.append(value)
#         return result

#     # Als het geen lijststring is, return lege lijst
#     return []


taken_df['tasks'] = taken_df['tasks'].apply(safe_eval)
taken_df = taken_df.explode('tasks')
tasks_normalized = pd.json_normalize(taken_df['tasks'])

tasks_normalized = tasks_normalized.rename(columns={
    'id': 'task_id',
    'address.latitude': 'latitude',
    'address.longitude': 'longitude',
    'timeWindow.from': 'from',
    'timeWindow.till': 'till'
})
tasks_normalized['id'] = taken_df['id'].values
tasks_normalized = tasks_normalized.reset_index(drop=True)

display(tasks_normalized.head())



In [ ]:
# dataframe met fixedTasks maken
fixed_taken_df = ingelezen_dataframe[['id', 'fixedTasks']].copy()
# filter om alleen rijen te tonen waar fixedTasks niet leeg is
fixed_taken_df = fixed_taken_df[fixed_taken_df['fixedTasks'].astype(str) != '[]']

fixed_taken_df['fixedTasks'] = fixed_taken_df['fixedTasks'].apply(safe_eval)
# fixedTasks omzetten van string naar object
fixed_taken_df = fixed_taken_df.explode('fixedTasks')

# alleen rijen behouden waar fixedTasks niet leeg is
fixed_taken_df = fixed_taken_df[fixed_taken_df['fixedTasks'].notna()]
fixed_taken_df = fixed_taken_df[fixed_taken_df['fixedTasks'] != {}]

# normaliseren van fixedTasks
fixed_tasks_normalized = pd.json_normalize(fixed_taken_df['fixedTasks'])

# kolommen hernoemen voor consistentie
if not fixed_tasks_normalized.empty:
    # voeg id kolom toe
    fixed_tasks_normalized['id'] = fixed_taken_df['id'].values
    fixed_tasks_normalized = fixed_tasks_normalized.reset_index(drop=True)

display(fixed_tasks_normalized.head())

In [ ]:
# Unieke locatie IDs toevoegen aan tasks_normalized
# Punten binnen 6 meter van elkaar krijgen dezelfde location_id

from sklearn.cluster import DBSCAN
import numpy as np

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Bereken de afstand tussen twee punten op aarde in meters
    """
    # Aardstraal in meters
    R = 6371000
    
    # Converteer naar radialen
    lat1_rad = np.radians(lat1)
    lat2_rad = np.radians(lat2)
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    
    # Haversine formule
    a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c
    
    return distance

# Bereid data voor clustering voor
coords = tasks_normalized[['latitude', 'longitude']].copy()

# Verwijder rijen met missende coördinaten
coords_clean = coords.dropna()

# Converteer naar radialen voor DBSCAN met haversine metric
coords_rad = np.radians(coords_clean[['latitude', 'longitude']].values)

# DBSCAN clustering
# eps in radialen: 6 meter / 6371000 meter (aardstraal) ≈ 0.000000941
eps_rad = 6 / 6371000  
dbscan = DBSCAN(eps=eps_rad, min_samples=1, metric='haversine')
clusters = dbscan.fit_predict(coords_rad)

# Voeg cluster labels toe aan de originele dataframe
tasks_normalized['location_id'] = np.nan
tasks_normalized.loc[coords_clean.index, 'location_id'] = clusters

# Converteer naar integer waar mogelijk
tasks_normalized['location_id'] = tasks_normalized['location_id'].astype('Int64')

# Toon resultaat
print(f"Aantal unieke locaties: {tasks_normalized['location_id'].nunique()}")
print(f"Aantal taken: {len(tasks_normalized)}")
display(tasks_normalized[['task_id', 'latitude', 'longitude', 'location_id']].head(20))

# Toon een voorbeeld van taken die dezelfde locatie delen
duplicated_locations = tasks_normalized[tasks_normalized.duplicated(subset=['location_id'], keep=False)]
if len(duplicated_locations) > 0:
    print(f"\nVoorbeeld van {len(duplicated_locations)} taken die locaties delen:")
    display(duplicated_locations.sort_values('location_id').head(10))

In [ ]:
# Voeg location_id toe aan fixed_tasks_normalized
# Match op basis van task_id (van fixed_tasks) en id (route id)

if not fixed_tasks_normalized.empty:
    # Check welke kolom in fixed_tasks_normalized de task id bevat
    # Dit zou 'taskId' of een vergelijkbare kolom moeten zijn
    print("Kolommen in fixed_tasks_normalized:")
    print(fixed_tasks_normalized.columns.tolist())
    print("\nKolommen in tasks_normalized:")
    print(tasks_normalized.columns.tolist())
    
    # Bepaal de juiste kolom naam voor task_id in fixed_tasks_normalized
    # Dit kan 'taskId', 'task_id', of een andere variant zijn
    task_id_col = None
    for col in fixed_tasks_normalized.columns:
        if 'task' in col.lower() and 'id' in col.lower():
            task_id_col = col
            break
    
    if task_id_col:
        # Merge om location_id toe te voegen
        # Match op zowel de task_id als de route id
        fixed_tasks_normalized = fixed_tasks_normalized.merge(
            tasks_normalized[['task_id', 'id', 'location_id']],
            left_on=[task_id_col, 'id'],
            right_on=['task_id', 'id'],
            how='left',
            suffixes=('', '_from_tasks')
        )
        
        # Verwijder de extra task_id kolom als die is aangemaakt
        if 'task_id' in fixed_tasks_normalized.columns and task_id_col != 'task_id':
            fixed_tasks_normalized = fixed_tasks_normalized.drop(columns=['task_id'])
        
        print(f"\nlocation_id toegevoegd aan fixed_tasks_normalized")
        print(f"Aantal fixed tasks met location_id: {fixed_tasks_normalized['location_id'].notna().sum()}")
        print(f"Aantal fixed tasks zonder location_id: {fixed_tasks_normalized['location_id'].isna().sum()}")
        
        display(fixed_tasks_normalized.head(10))
    else:
        print("Kan geen task_id kolom vinden in fixed_tasks_normalized")
        display(fixed_tasks_normalized.head())
else:
    print("fixed_tasks_normalized is leeg")

In [ ]:
display(ingelezen_dataframe.head())
display(tasks_normalized.head())
display(fixed_tasks_normalized.head())